# Gain scheduling and Auto tuning
In this notebook the topic of gain scheduling is studied.
## Preparation
Study the relevant chapters in
- "Predictive and Adaptive Control" R. Johansson 
- Chapter 9 "Adaptive Control" K.J. Åström
- [Slides from control system synthesis](http://www.control.lth.se/media/Education/DoctorateProgram/2016/Control%20System%20Synthesis/gainscheduling.pdf)

## The processes
The processes studied in this session are third order systems with two poles in $s=-0.5$ and one pole in -2.
$$P(s) = \dfrac{1}{(2s+1)^2(0.5s+1)}$$
with or without input nonlinearity.

## Installation
- First of all, you need Julia installed, see https://julialang.org/downloads/
- You also need [IJulia](https://github.com/JuliaLang/IJulia.jl#installation), which in turn requires Jupyter.
- Once Julia is up and running, the following julia code only has to be run once, it installs the relevant packages.

The code below must be run once every time the kernel is restarted

In [ ]:
using ControlSystems, OrdinaryDiffEq, Plots, Interact, Pkg, LinearAlgebra, Statistics
gr(show=false, size=(800,400)) # Set defaults for plotting

We now proceed to define the process model and some parameters for our simulation

In [ ]:
P              = tf(1,[2.,1])^2*tf(1,[0.5,1])  # Process model
h              = 0.1             # Sample time (only used for plots)
Ps             = ss(P)           # State-space representation of process model
reference(x,t) = 1.;             # Reference generator (step function)

Next we define a simulation object and simulate the system with a P-controller.
- Change the value of the gain $K$ using the slider and try to get a feeling for how the system behaves.

In [ ]:
s      = Simulator(Ps)
x0     = [0.,0,0] # Initial state
Tf     = 20              # Length of experiments (seconds)
t      = 0:h:Tf          # Time vector
tspan  = (0.0,Tf)

In [ ]:
@manipulate for K = LinRange(0,20,50)
    global os, sol1
    controller(x,t) = K*(reference(x,t) .- Ps.C*x)
    os     = Simulator(Ps,controller)
    sol1   = solve(os, x0, tspan, Tsit5()) # Simulate the system
    plot(t, os.y(sol1, t)[:], lab="P controller")
    plot!([tspan...], [1,1], lab="Reference", l=:dash, c=:black, ylims=(0,1.5))
end

- As you may notice, the stationary error never vanishes. How can one obtain zero stationary error for a fixed reference value while keeping a P controller?
- Modify the function `controller` to get a smaller stationary error.

Let's step up the game and control the system with a PID controller. Use [Ziegler–Nichols frequency method](https://en.wikipedia.org/wiki/Ziegler%E2%80%93Nichols_method) to tune a PID controller for the system. 
You can use the code above to find the critical gain. You can compare the result with a numerical root finding algorithm applied to the process model, see below
- Compare the results with manual tuning.

In [ ]:
using Roots
fzero(K->maximum(real.(pole(feedback(P,tf(K))))), 1, 20)

Below we show the root locus of the system, what does the plot tell you?

In [ ]:
rlocusplot(P); plot!([0,0],[-5,5],l=(:dash,:black),lab="")

In [ ]:
conditions  = [(x,y,r) -> true]
Tf          = 20              # Length of experiments (seconds)
t           = 0:h:Tf          # Time vector
tspan       = (0.0,Tf)
x0          = [0.,0,0]
feedforward = [0]

@manipulate for kp = LinRange(0,10,50),ki = LinRange(0,5,50),kd = LinRange(0,20,50)
    controllers = [pid(kp=kp, ki=ki, kd=kd)]
    global gs, sol1, sol2
    gs    = GainSchedulingSimulator(Ps, reference, controllers, conditions, feedforward=feedforward)
    sol2  = solve(gs, x0, tspan, Tsit5())
    plot(t, os.y(sol1, t)[:], lab="P controller")
    plot!(t, gs.y(sol2, t)[:], lab="PID controller")
    plot!([tspan...], [1,1], lab="Reference", l=(:dash, :black), ylims=(-0.1,1.5))
end

## Auto tuning
Even with interactive tools, tuning a PID controller by hand can be difficult. A basic control course teaches us how to design a PID controller based on, e.g., pole placement. Here, we will formulate an optimization problem where we optimize a cost function $J(k_p,k_i,k_d)$ based on the step response of the closed-loop system
$$J(k_p,k_i,k_d) \quad=\quad \dfrac{1}{T_f}\int_0^{T_f} |r(t)-y(t)| dt \quad=\quad \dfrac{1}{T_f}\int_0^{T_f} |e(t)|dt$$

In [ ]:
Pkg.add("NLopt") # Install NLopt if not already installed

In [ ]:
using NLopt, ForwardDiff
p              = [0.1, 0.1, 0.1] # Initial guess [kp, ki, kd]
Kpid(kp,ki,kd) = pid(kp=kp, ki=ki, kd=kd)
Kpid(p)        = K(p...)

function timedomain(p)
    C     = Kpid(p[1], p[2], p[3])
    L     = feedback(P*C) |> ss
    s     = Simulator(L, (x,t) -> [1]) # Sim. unit step load disturbance
    ty    = eltype(p) # So that all inputs to solve have same numerical type (ForwardDiff.Dual)
    x0    = zeros(L.nx) .|> ty
    tspan = (ty(0.),ty(Tf))
    sol   = solve(s, x0, tspan)
    y     = L.C*sol(t) # y = C*x
    y
end

function costfun(p)
    y = timedomain(p)
    mean(abs,1 .- y) # ~ Integrated absolute error IAE
end

f_cfg = ForwardDiff.GradientConfig(costfun, p)

function f(p::Vector, grad::Vector)
    if length(grad) > 0
        grad .= ForwardDiff.gradient(costfun,p,f_cfg)
    end
    costfun(p)
end

function runopt(p; f_tol = 1e-5, x_tol = 1e-3)
    opt = Opt(:LD_AUGLAG, 3)
    lower_bounds!(opt, 1e-6ones(3))
    xtol_rel!(opt, x_tol)
    ftol_rel!(opt, f_tol)

    min_objective!(opt, f)
    NLopt.optimize(opt, p)[2]
end

@info "Starting Optimization"
@time p = runopt(p, x_tol=1e-6)
y = timedomain(p)
plot(t,y')

If we look at the sensitivity function of the closed-loop system obtained with the optimized controller, we see that the optimization procedure produced a quite aggressive controller with poor robustness. 

In [ ]:
C     = Kpid(p...)
gangoffourplot(P,C, exp10.(LinRange(-1,3,500)), legend=false)

To combat this, we can add bounds on the maximum value of the sensitivity functions:

In [ ]:
Ω  = exp10.(LinRange(-1,2,150))
p0 = [0.1,0.1,0.1]
function freqdomain(p)
    C     = Kpid(p[1], p[2], p[3])
    S     = 1/(1+P*C) # Sensitivity fun
    T     = tf(1.) - S# Comp. Sensitivity fun
    Sw    = vec(bode(S,Ω)[1]) # Freq. domain constraints
    Tw    = vec(bode(T,Ω)[1]) # Freq. domain constraints
    Sw,Tw
end
Mt = 1.5
@manipulate for Ms = LinRange(1.1,2,20)
    global p0, p
    p = p0
    function constraintfun(p)
        Sw,Tw = freqdomain(p)
        [maximum(Sw)-Ms; maximum(Tw)-Mt]
    end

    g_cfg = ForwardDiff.JacobianConfig(constraintfun, p)

    function c(result, p::Vector, grad)
        if length(grad) > 0
            grad .= ForwardDiff.jacobian(constraintfun,p,g_cfg)'
        end
        result .= constraintfun(p)
    end

    function runopt(p; f_tol = 1e-5, x_tol = 1e-3, c_tol = 1e-8)
        opt = Opt(:LD_SLSQP, 3)
        lower_bounds!(opt, 1e-6ones(3))
        xtol_rel!(opt, x_tol)
        ftol_rel!(opt, f_tol)

        min_objective!(opt, f)
        inequality_constraint!(opt, c, c_tol*ones(2))
        NLopt.optimize(opt, p)[2]
    end


    p = runopt(p, x_tol=1e-6)
    y = timedomain(p)
    Sw,Tw = freqdomain(p)
    plot(t,y', layout=2, show=false, ylims=(0,1.3))
    plot!(Ω, [Sw Tw] , lab=["Sw" "Tw"], subplot=2, xscale=:log10, yscale=:log10)
    plot!([Ω[1],Ω[end]], [Ms,Ms], c = :black, l=:dash, subplot=2, lab="Ms")
    plot!([Ω[1],Ω[end]], [Mt,Mt], c = :purple, l=:dash, subplot=2, lab="Mt", ylims=(0.01,3))
end

In [ ]:
C     = Kpid(p...)
gangoffourplot(P,C, exp10.(LinRange(-1,3,500)), legend=false)

- Does introduction of a penalty on the maximum sensitivity make the controller more robust?

## Gain Scheduling 

We now introduce gain scheduling. The following code defines two controllers and a vector of conditions that determine when to switch between the controllers. The switching conditions are allowed to be arbitrary functions of the state $x$, the output $y$ or the reference $r$. The template below implements a schedule based on a simple threshold value on the output, $y=th$. Experiment with the controller parameters of the two controllers and the scheduling signal. Anser the following questions
- Which signal is best for scheduling?
- Can you implement a controller which is both faster and has better damping than your best controller from the previous task?

In [ ]:
feedforward = Float64[0,0]
@manipulate for kp1 = LinRange(0,20,50),ki1 = LinRange(0,5,50),kp2 = LinRange(0,20,50),ki2 = LinRange(0,5,50),kd  = LinRange(0,20,50),th  = LinRange(0,1,20)
    global gs
    plot(t, gs.y(sol2, t)[:], lab="PID controller")

    controllers  = [pid(kp=kp1, ki=ki1, kd=kd), pid(kp=kp2, ki=ki2, kd=kd)]
    conditions   = [(x,y,r) -> y[1] < th, (x,y,r) -> y[1] >= th]
    gs2          = GainSchedulingSimulator(Ps, reference, controllers, conditions, feedforward=feedforward)
    sol3         = solve(gs2, x0, tspan, Tsit5())
    plot!(t, gs2.y(sol3, t)[:], lab="Gain scheduled controller", ylims=(0,1.5))
    plot!([tspan...], [1,1], lab="Reference", l=(:dash, :black))
    plot!([tspan...], [th, th], lab="Threshold", l=(:dash, :red), legend=:bottomright)
end

Okay, now it's time to make it a bit harder. We now add a nonlinearity to the process input. We can play around a bit with the type of the nonlinearity, but let's start with
$$u = u_i^2 \quad \text{if}\quad  u_i > 0$$
$$u = 0.1u_i \quad \text{otherwise}$$
depicted below.

In [ ]:
nonlinearity(u) = u < 0 ? 0.1u : u^2
plot(LinRange(-1,1.5,200),nonlinearity,lab="Input nonlinearity")

Try tuning a P-controller for the system with the input nonlinearity. Is it possible to achieve an okay result?

In [ ]:
Tf             = 75.
tspan          = (0.,Tf)
t              = 0:h:Tf
x0             = [0,0,5/2]
conditions     = [(x,y,r) -> true]
reference(x,t) = t < Tf÷2 ? 0. : 1.
feedforward    = Float64[0]
@manipulate for kp = LinRange(0,10,50),ki = LinRange(0,8,50),kd = LinRange(0,20,50)
    global controllers
    controllers = [pid(kp=kp, ki=ki, kd=kd)]
    global gs, sol2
    gs    = GainSchedulingSimulator(Ps, reference, controllers, conditions,
        inputfun=(u,t)->nonlinearity.(u), feedforward=feedforward)
    sol2  = solve(gs, x0, tspan, Tsit5())
    plot(t, gs.y(sol2, t)[:], lab="PID controller")
    plot!(t, reference.(0,t), lab="Reference", l=:dash, c=:black, ylims=(-0.5,2.))
end

Once again, we try a controller with gain scheduling. The left plot displays the control signal of the two controllers.

In [ ]:
gs    = GainSchedulingSimulator(Ps, reference, controllers, conditions, inputfun=(u,t)->nonlinearity.(u) .+ disturbance(t,0))
Tf    = 75.
tspan = (0.,Tf)
t     = 0:h:Tf
feedforward = Float64[0,1]

reference(x,t) = t < Tf÷3 ? 0. : 1.
disturbance(x,t) = 0 # t < 2Tf÷3 ? 0. : -1.
@manipulate for kp1 = LinRange(0,20,50),ki1 = LinRange(1e-5,8,50),kd1 = LinRange(1e-5,10,50),kp2 = LinRange(0,10,50),ki2 = LinRange(1e-5,8,50),kd2 = LinRange(1e-5,8,50),th  = LinRange(0,1,20)
    
    # kp1,ki1,kp2,ki2,th = 1,1,1,1,0.5
    global sol3
    sol2  = solve(gs, x0, tspan, Tsit5())
    plot(t, gs.y(sol2, t)[:], lab="PID controller", layout=2)

    controllers  = ss.([pid(kp=kp1, ki=ki1, kd=kd1), pid(kp=kp2, ki=ki2, kd=kd2)])
    conditions   = [(x,y,r) -> y[1] < th, (x,y,r) -> y[1] >= th]
    gs2          = GainSchedulingSimulator(Ps, reference, controllers, conditions,
                        inputfun=(u,t)->nonlinearity.(u) .+ disturbance(0,t), feedforward=feedforward)
    sol3         = solve(gs2, x0, tspan, Tsit5(), reltol=1e-4, abstol=1e-4)

    # Plotting
    U  = hcat(sol3.u...)
    x  = [[u[1],u[2],u[3]] for u in sol3.u]
    u1 = controllers[1].C*U[3:4,:] + controllers[1].D.*hcat((gs2.e.(x, sol3.t))...)
    u2 = controllers[2].C*U[5:6,:] + controllers[2].D.*hcat((gs2.e.(x, sol3.t))...)
    plot!(t, gs2.y(sol3, t)[:], lab="Gain scheduled controller", ylims=(-0.5,2), subplot=1)
    plot!(t, reference.(0,t), lab="Reference", l=:dash, c=:black, subplot=1)
    plot!([tspan...], [th, th], lab="Threshold", c=:red, l=:dash, subplot=1)
    plot!([u1[:] u2[:]], lab=["u_1" "u_2"], subplot=2)
end

- Can you manage to get a gain scheduled controller that is better than the simple PID controller?
- Add feedforward if needed
- Add an input disturbance by uncommenting "# t < 2Tf÷3 ? 0. : -1.". Does your scheduling still work as well?
- Try setting $k_i$ and $k_d$ to zero to use two proportional controllers and play around with the threshold value. Can you manage to get the controller stuck in the middle of the step response?

To get a better feeling for the practicalities of gain scheduling, you'll be asked to implement a simple gain scheduled controller with interpolation below. We consider the same process and nonlinearity as above, but this time we'll implement two P-controllers and an interpolation region.
- Define two threshold values $t_1$ and $t_2$, between which you interpolate the value of the control signal requested by the two controllers.
- Figure out the equations for linear interpolation between the thresholds, you have two candidate control signals $u_1,u_2$, what is the control signal to output in the following cases, where $p$ is the scheduling signal?
    1. $p < t_1$
    2. $t_1 \leq p < t_2$
    3. $p \leq t_2$
Add the interpolation logic to the controller below.

In [ ]:
Tf             = 50                 # Length of experiments (seconds)
t              = 0:h:Tf             # Time vector
reference(x,t) = t < Tf÷2 ? 0. : 1. # Reference generator (step function)
s              = Simulator(Ps)
x0             = [0.,0,5/2]         # Initial state
tspan          = (0.0,Tf)

@manipulate for K1 = LinRange(0,10,50), K2 = LinRange(0,5,50)

    function controller(x,t)
        y  = Ps.C*x # Form measurement signal
        e  = reference(x,t) .- y
        u1 = K1*e
        u2 = K2*e
        
        u  = (u1+u2) / 2 # Implement the interpolation here! <-----------------------------

        return u
    end
    
    s    = Simulator(Ps,(u->nonlinearity.(u)) ∘ controller)
    solg = solve(s, x0, tspan, Tsit5()) # Simulate the system
    plot(t, s.y(solg, t)[:], lab="GS controller", layout=2, subplot=1)
    plot!(t, reference.(0,t), lab="Reference", l=:dash, c=:black, ylims=(-0.5,1.5), subplot=1)
    plot!(t, [controller(solg(t),t)[] for t ∈ t], title="u", ylims=(-8,8), subplot=2)
end

You can add sliders for $t_1$ and $t_2$ to make experimentation faster.
- Which signal is your preferred choice of scheduling signal?

Tuning a gain schedule and the parameters of the scheduled controllers can become very tedious. Even more so it the process time constant is long and one does not have a simulator. A more systematic apporach is to linearize the system around two different operating points and design a well-damped controller for both linear systems. One then chooses a threshold somewhere between the operating points. One can alternatively use the autotuning approach to tune the controller based on several different responses, to bot reference changes and load disturbances.